In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
teen_mental_health_df=pd.read_csv('/content/final_teen_mental_health_chatbot_dataset.csv')
print(teen_mental_health_df.head())
print("**************************")
print(f"The shape of the dataset is : {teen_mental_health_df.shape}")

In [ ]:
teen_mental_health_df.info()

In [ ]:
print(teen_mental_health_df.isnull().sum())

# **Text Preprocessing**
* Converting to lowercase
* Removal of Stopwords
* Lemmatization  

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(teen_mental_health_df['user_input']))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for User Input ')
plt.show()
print("*"*80)
print("-"*80)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(teen_mental_health_df['bot_response']))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Bot Response ')
plt.show()

In [ ]:
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r"\d+", "", text)
    tokens = word_tokenize(text)
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return " ".join(cleaned_tokens)


teen_mental_health_df["cleaned_user_input"] = teen_mental_health_df["user_input"].apply(preprocess_text)
teen_mental_health_df["cleaned_bot_response"]=teen_mental_health_df["bot_response"].apply(preprocess_text)

print(teen_mental_health_df[["user_input", "cleaned_user_input"]].head())


In [ ]:
print(teen_mental_health_df['cleaned_bot_response'].head(20))

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.probability import FreqDist
def frequent_words(df, col, top_n=20):
    all_words = []
    for text in df[col]:
        words = word_tokenize(text)
        all_words.extend(words)

    word_freq = FreqDist(all_words)
    return word_freq.most_common(top_n)

In [ ]:
def plot_frequent_words(df, col, top_n=20):
    top_words = frequent_words(df, col, top_n)
    words, counts = zip(*top_words)

    freq_df = pd.DataFrame({'Word': words, 'Frequency': counts})

    plt.figure(figsize=(12, 6))
    sns.barplot(x='Frequency', y='Word', data=freq_df, palette='viridis')
    plt.title(f'Top {top_n} Most Frequent Words')
    plt.xlabel('Frequency')
    plt.ylabel('Words')
    plt.tight_layout()
    plt.show()

plot_frequent_words(teen_mental_health_df, 'cleaned_user_input', top_n=20)

In [ ]:
plot_frequent_words(teen_mental_health_df, 'cleaned_bot_response', top_n=20)


In [ ]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures


all_tokens = []
for text in teen_mental_health_df['cleaned_user_input']:
    all_tokens.extend(word_tokenize(text))


bigram_finder = BigramCollocationFinder.from_words(all_tokens)
top_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)


trigram_finder = TrigramCollocationFinder.from_words(all_tokens)
top_trigrams = trigram_finder.nbest(TrigramAssocMeasures.likelihood_ratio, 10)

print("Top Bigrams (Collocation):", top_bigrams)
print("Top Trigrams (Collocation):", top_trigrams)

In [ ]:
# Create an NLTK Text object
text_obj = nltk.Text(all_tokens)

# Show concordance for a keyword (e.g., 'feel')
print(text_obj.concordance("friend", width=60, lines=10))


# **Sentiment Analysis**

# Sentiment Analysis Using Vader
* Uses a bag of words approach
* It gives a compound score , negative , positive and neutral scores


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
teen_mental_health_df['user_sentiment'] = teen_mental_health_df['cleaned_user_input'].apply(lambda x: analyzer.polarity_scores(x)['compound']>0 and 'Positive' or analyzer.polarity_scores(x)['compound']<0 and 'Negative' or 'Neutral')

In [ ]:
print(teen_mental_health_df['user_sentiment'])

In [ ]:
teen_mental_health_df.groupby('user_sentiment')['user_input'].count().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Distribution of User Sentiment')
plt.show()

In [ ]:
res = {}
from tqdm.notebook import tqdm

for i, row in tqdm(teen_mental_health_df.iterrows(), total=len(teen_mental_health_df)):
    try:
        text = row['cleaned_bot_response']
        res[i] = analyzer.polarity_scores(text)
    except RuntimeError:
        pass

print(res)


In [ ]:
vaders_df = pd.DataFrame.from_dict(res, orient='index')
teen_mental_health_df = teen_mental_health_df.join(vaders_df,how='left')
print(teen_mental_health_df.head())

In [ ]:
ax=sns.barplot(data=teen_mental_health_df,x='user_sentiment',y='compound')
ax.set_title('Compound Sentiment')
plt.show()

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(15,5))
sns.histplot(vaders_df['pos'],ax=axs[0])
sns.histplot(vaders_df['neg'],ax=axs[1])
sns.histplot(vaders_df['neu'],ax=axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Negative')
axs[2].set_title('Neutral')
plt.tight_layout()
plt.savefig('sentiment.jpg')
plt.show()

 1. **Positive Sentiment Distribution**
> Most scores are between 0.0 and 0.2, indicating that the majority of bot responses have low positivity.

> A small number of responses have scores above 0.6, suggesting some strongly positive messages, but they are relatively rare.

 **Interpretation**:
* The bot generally maintains a low-key supportive tone
* It responds with few highly enthusiastic or cheerful messages — possibly to remain empathetic rather than overly cheerful.


---



 2. **Negative Sentiment Distribution**
> A large peak at 0.0–0.1: most responses have very low negativity.

> Only a few responses exceed 0.4–0.5, and almost none go beyond 0.7.

 **Interpretation**:
`The chatbot is carefully avoiding negative language, which is expected and appropriate in mental health contexts.`


---



3. **Neutral Sentiment Distribution**
> The scores are more evenly spread, with a peak around 0.5, and many values between 0.3 to 0.7.

> Some responses are highly neutral (0.9–1.0) — possibly more factual or non-emotional.

**Interpretation**:
`The chatbot responses are primarily neutral in tone, which aligns with maintaining balance, objectivity, and emotional safety during mental health conversations.`


---


**Sentiment	Key Insights**
1. **Negative** : 	Most responses are non-negative, as expected in a supportive context.
2. **Neutral**: 	The chatbot relies heavily on neutral language, which is often seen in calming or informational responses.
3. **Positive	Positivity** is present but generally mild — the bot avoids sounding overly upbeat, possibly to stay relatable.



---





> One issue with vader sentiment analyzer is that it doesn't take context into account

> So we will compare vaders result to Roberta Pre Trained model which accounts for context too .

# Using Roberta For Sentiment Analysis

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
def polarity_scores_roberta(example):
    encoded_text=tokenizer(example,return_tensors='pt')
    output=model(**encoded_text)
    scores=output[0][0].detach().numpy()
    scores=softmax(scores)
    scores_dict={
        'roberta_neg':scores[0],
        'roberta_neu':scores[1],
        'roberta_pos':scores[2]
    }
    return scores_dict

In [ ]:
for i,row in tqdm(teen_mental_health_df.iterrows(),total=len(teen_mental_health_df)):
    try:
        text=row['cleaned_bot_response']
        roberta_result=polarity_scores_roberta(text)
        for key,value in roberta_result.items():
            teen_mental_health_df.loc[i,key]=value
    except RuntimeError:
        pass
print(teen_mental_health_df.head())

In [ ]:
sns.pairplot(data=teen_mental_health_df,vars=['neg','neu','pos','roberta_neg','roberta_neu','roberta_pos'],hue='user_sentiment',palette='tab10')
plt.savefig('Roberta_Vs_Vader.jpg')
plt.show()

***Key Inferences from the Plot***
1. **VADER vs RoBERTa Distributions**

> VADER scores (neg, neu, pos) are mostly well spread and mutually exclusive (neg vs pos shows a clear inverse relationship).

> RoBERTa scores are more concentrated and less sharply defined — more probabilistic than rule-based VADER.


---



2. **Diagonal KDE plots**

`The diagonal plots show density distributions of each sentiment score`:

* VADER pos has a wide distribution, with Positive-labeled points peaking at higher values.

* RoBERTa pos shows a narrow, steep spike: it's more confident and binary, which is typical of deep learning models.

* VADER neu is flatter; RoBERTa neu shows sharper bimodality — may reflect stronger neutral decisions.


---


3. **Off-Diagonal Scatter Plots**
 > **neg vs pos and roberta_neg vs roberta_pos**:

* Clearly negatively correlated, as expected — when negative is high, positive is low.

* RoBERTa shows more saturation, i.e., values near 0 or 1, reflecting high model confidence.


---


**VADER and RoBERTa comparisons**:

> VADER pos and RoBERTa pos have some agreement but are not perfectly correlated.

> Similarly for neg — RoBERTa may classify strong negations more sharply than VADER.


---



In [ ]:
sns.heatmap(teen_mental_health_df[['neg', 'neu', 'pos', 'roberta_neg', 'roberta_neu', 'roberta_pos']].corr(), annot=True)
plt.show()

In [ ]:
fig,axs=plt.subplots(1,3,figsize=(15,5))
sns.boxplot(x='user_sentiment',y='neg',data=teen_mental_health_df,ax=axs[0])
sns.boxplot(x='user_sentiment',y='roberta_neg',data=teen_mental_health_df,ax=axs[1])
sns.boxplot(x='user_sentiment',y='neu',data=teen_mental_health_df,ax=axs[2])

axs[0].set_title('Negative')
axs[1].set_title('Roberta Negative')
axs[2].set_title('Neutral')

plt.tight_layout()
plt.show()

fig,axs=plt.subplots(1,3,figsize=(15,5))
sns.boxplot(x='user_sentiment',y='roberta_neu',data=teen_mental_health_df,ax=axs[0])
sns.boxplot(x='user_sentiment',y='pos',data=teen_mental_health_df,ax=axs[1])
sns.boxplot(x='user_sentiment',y='roberta_pos',data=teen_mental_health_df,ax=axs[2])
axs[0].set_title('Roberta Neutral')
axs[1].set_title('Positive')
axs[2].set_title('Roberta Positive')
plt.tight_layout()
plt.show()


1. **VADER Negative (neg)**

> `Median neg score is higher for user-labeled Negative responses compared to others`.

> `But some Positive-labeled responses also have high neg scores → VADER can misclassify certain emotional language.`

> `Distribution overlaps across all labels → moderate separation.`


---



2. **RoBERTa Negative (roberta_neg)**

> `Clearer separation than VADER`:

> `Highest medians for Negative-labeled data.`

> `Positive and Neutral have much lower scores with tighter ranges.`

> `Outliers for Positive and Neutral → some misclassifications, but generally well-separated.`


---


*  **Inference**: `RoBERTa is more confident and effective at detecting negative sentiment than VADER.`


---



3. **VADER Neutral (neu)**

> `All sentiment classes have similarly high neu scores.`

> `Median values are tightly clustered → VADER overuses neutral scores, making it less discriminative.`

**Issue:** VADER may over-rely on neutrality, especially in emotionally complex messages.


---


4. **RoBERTa Neutral (roberta_neu)**

> `Still shows some overlap, but better spread across sentiment classes.`

> `Slightly lower median for Positive compared to Negative and Neutral.`


---



* **Inference:** `RoBERTa neutral scores are more balanced and offer slightly better class differentiation than VADER.`


---


5. **VADER Positive (pos)**

> `Positive-labeled responses have higher pos scores than others, as expected.`

> `However, overlap remains — Negative and Neutral responses also show non-negligible pos scores.`

> `VADER may detect surface-level positivity (e.g., “I’m okay”) even in negative contexts.`


---



6. **RoBERTa Positive (roberta_pos)**

> `Excellent separation`

> `Positive-labeled responses have very high positive scores.`

> `Negative and Neutral have low medians, especially Negative.`

> `Few outliers in Negative → minimal noise.`


---


* **Inference**: `RoBERTa is highly confident and effective at recognizing positive sentiment.`


---



# Final Insights

**Part 1: Mental Health of teenagers
Using sentiment analysis (VADER + RoBERTa) on children's user_input, we can draw the following inferences**:

 1. *Predominantly Neutral or Negative Tone*

`Many inputs were classified as neutral or negative, indicating`:

> A sense of emotional numbness, confusion, or resignation

> Presence of stress, sadness, anxiety, or loneliness

> High neutrality can also mean they mask emotions or use non-expressive language, which is common in adolescents dealing with mental stress.


---


2. *Subtle Positive Inputs*

> Fewer inputs had strong positive sentiment.

> Even “I'm fine” or “okay” might be false positives, as emotionally distressed users often understate their feelings.


---



3. *Topics of Concern (from Bigrams/Trigrams)*
`A phrase frequency analysis (like "feel alone", "no friends", "not happy"), it likely reinforces`:

> Social isolation

> Academic or parental pressure

> Identity or mental health struggles


---


**Conclusion**: Many children in the dataset show low or mixed emotional valence, consistent with symptoms of emotional suppression, depression, or mild to moderate distress.


---


---




**Part 2: Effectiveness of Bot Responses
The bot's responses were analyzed using VADER and RoBERTa . Here's what we can infer**:

1. *Responses Skew Heavily Neutral*

> Most bot responses have high neutral scores and low negative scores, which is good for avoiding harm but may seem robotic or impersonal.


---


2. *Moderate Positive Sentiment*

> Some responses are gently positive, which is helpful — bots should aim for empathetic encouragement, not toxic positivity.

> RoBERTa identified a few strongly positive responses — this suggests the bot does try to uplift the user, but selectively.


---


3. *Lack of Deep Emotional Mirroring*

`If users are expressing genuine distress (as shown by negative or neutral inputs), the bot's overly-neutral replies may feel emotionally mismatched.`

> Good therapy bots often use mirroring + validating language like:

`“That sounds really difficult. I'm here with you.”`



---


**Conclusion**:
`The bot is safe and non-triggering, but could be improved by':

> Using slightly more empathetic language

> Offering active listening cues or follow-up prompts

> Matching the emotional tone of user inputs more closely



---



---

